# 0. For the moment, communicating with localhost 

# 1. Save CKKS parms, keys, and Ciphertexts in C++

# 2. Load them in Python

# 3. Do a quick computation 

# 4. Save Ciphertext in a appropriate form 
to be sent over TCP/IP and then read by SEAL C++ 

In [6]:
import numpy as np

import fase

from fase import seal

from io import StringIO, BytesIO

In [2]:
from fase.core.common import HEAANContext, HEAANParameters, SEALContext

# SEAL setup
poly_modulus_degree = 16384
coeff_moduli = [37, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 37]

ckks_se = SEALContext(poly_modulus_degree=poly_modulus_degree,
                             coeff_moduli=coeff_moduli,
                             scale_bit=28)

SEAL CKKS scheme is ready


In [3]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)

In [4]:
ckks_se.parms.save("prams.sav")

## Ciphertext를 파일로 저장했다가 파일에서 로드하는거 가능

In [5]:
sk = ckks_se.sk

In [6]:
enc = ckks_se.encrypt([830422])

fn_out = './enc.dat'
enc.save(fn_out)

## 파라미터를 파일로 저장했다가 파일에서 로드 가능

In [26]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)

In [27]:
parms.load('./prams.sav')

In [29]:
parms.poly_modulus_degree()

16384

## 파일을 socket으로 전송
### Qt에서 자동으로 call 하도록 변경

In [1]:
import numpy as np 

def smoothed_frame_N(scene, nframe = 10, window_size = 5, shift=0):
    FPS_ORIGINAL=10 # 인 것 같음...
    
    nskip = int((len(scene)-shift) / nframe)
    #nframe = np.ceil((len(scene) - window_size)/ nskip).astype(int)

    sub = np.zeros(nframe, dtype=scene.dtype) #scene.dtype - frame

    for i in range(nframe):
        temp = scene[i*nskip+shift:i*nskip+window_size+shift]

        for feat in temp.dtype.names: # recarry라서 한 번에 np.mean 불가능
            sub[i][feat] = np.median(temp[feat])

    return sub


def ravel_rec(sub, return_feature=False):
    vec=[]
#     if return_feature:
#         features=[]
#         for line in sub:
#             this_line=[]
#             for ff in line.dtype.names:
#                 if ff not in "frame":
#                     this_line.append(line[ff])
#                     features.append(ff)
#             vec.extend(this_line)
                    
#         return features, np.array(vec)
#     else:
    for line in sub:
        vec.extend([line[ff] for ff in line.dtype.names if ff not in "frame"])

    return np.array(vec)

In [2]:
import kinect_utils as ku
import pickle

def do_preprocess(point_pickle):
    #point_pickle = "/home/hoseung/Work/Kinect/pykinect_BBS/G1/000/BT/bodytracking_data.pickle"
    klist = pickle.load(open(point_pickle, "rb"))

    scene = ku.kinect2mobile_direct(klist)

    nframe = 10 
    shift=0
    sub = smoothed_frame_N(scene, nframe=nframe, shift=shift)
    rav_sub = ravel_rec(sub)[np.newaxis, :]
    pickle.dump(rav_sub, open("rav_example.pickle", "wb"))

In [13]:
import tqdm
import socket
import os
import time

SEPARATOR = "<sep>"
fn_out = ['whole_rf.pickle','./enc.dat',
          "/home/hoseung/Work/Kinect/pykinect_BBS/G1/000/BT/bodytracking_data.pickle",
          "/home/hoseung/Work/fhenrf/pose/rav_example.pickle"][3]

host = ['10.100.82.54', '10.100.82.55', socket.gethostname()][2]

point_pickle = "/home/hoseung/Work/Kinect/pykinect_BBS/G1/000/BT/bodytracking_data.pickle"


while not os.path.exists(point_pickle):
    time.sleep(1)
    
do_preprocess(point_pickle)

In [20]:
port = 9999

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# AF_INET = IPv4 

s.connect((host, port))

#BUFFER_SIZE=4096
END_MARKER="END"

In [21]:
filesize = os.path.getsize(fn_out)
BUFFER_SIZE = filesize

s.send(f"{fn_out}{SEPARATOR}{filesize}".encode())
s.send(f"{fn_out}{SEPARATOR}{filesize}".encode())

# start sending the file
progress = tqdm.tqdm(range(filesize), f"Sending {fn_out}", unit="B", unit_scale=True, unit_divisor=1024)
with open(fn_out, "rb") as f:
    while True:
        # read the bytes from the file
        bytes_read = f.read(BUFFER_SIZE)
        if not bytes_read:
            # file transmitting is done
            #s.sendall(bytes(END_MARKER, "utf-8"))
            break
        # we use sendall to assure transimission in 
        # busy networks
        s.sendall(bytes_read)
        # update the progress bar
        progress.update(len(bytes_read))
#server_message = s.recv(1024).decode()

Sending /home/hoseung/Work/fhenrf/pose/rav_example.pickle: 100%|████████████| 2.49k/2.49k [00:17<00:00, 144B/s]


In [19]:
s.close()

In [6]:
import socket
s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
LOCALHOST = '127.0.0.1'
port = 9990

s.connect((LOCALHOST,port))
print("New client created:")

while True:
    client_message = input("Me: ")
    s.sendall(client_message.encode())
    
    
    filesize = os.path.getsize(fn_out)
    BUFFER_SIZE = filesize
    s.send(f"{fn_out}{SEPARATOR}{filesize}".encode())
    with open(fn_out, "rb") as f:
        print("sending... ")
        while True:
            # read the bytes from the file
            bytes_read = f.read(BUFFER_SIZE)
            if not bytes_read:
                # file transmitting is done
                #s.sendall(bytes(END_MARKER, "utf-8"))
                break
            # we use sendall to assure transimission in 
            # busy networks
            s.sendall(bytes_read)

    msg_received = s.recv(1024)
    msg_received = msg_received.decode()
    print("Server:",msg_received)

    if msg_received == 'exit':
        s.sendall("ByeBye".encode())
        break;
s.close()


New client created:
Me: 1
sending... 


KeyboardInterrupt: 

##### 